# Quick Start

In this notebook:
* Setup ApertureDB database server 
* Setup a Google Colab notebook with aperturedb client
* Then configure it
* Load a dataset ApertureDB server
* Query!

## 1. Set up the Database

ApertureDB is set up as a database (server) and can be accessed from clients anywhere as long as the server is accessible on the network to the client.

Sign up for an Aperture [cloud account here](aperturedata.io/cloud) (30 days free trial) or see [other methods here](http://docs.aperturedata.io/category/setup-aperturedb)

## 2. Set up the Client

### a) Launch Colab or Jupyter Server

* Run any of our notebooks on Google Colab or your own Jupyter server as indicated from Step 2.b) onwards.
* You can use our [pre-configured Jupyter server](https://docs.aperturedata.io/Setup/client/notebooks) (launches on port 8888) which will already have the SDK installed (so you can skip Step 2.b)) by doing:
  
   **docker run --interactive --tty aperturedata/aperturedb-notebook**

### b) Install ApertureDB SDK

In [ ]:
! pip install -q aperturedb

### c) Define server configuration

**On ApertureDB-Cloud**

When you click on Connect for your instance, you can copy the JSON configuration string for your server. It looks like:

```javascript
{
  "host": "localhost",
  "username": "admin",
  "password": "admin",
  .....
}
```

For any other method, the parameters for host will depend on where the database is hosted, default username and password is admin:admin, and the default port that ApertureDB listens on is 55555 (except on Macbooks where we choose a different port like 55557). 

### d) For Colab - Add Server Configuration to Secrets

The easiest way to configure one or more Colab notebooks to point to the same ApertureDB server is to put the JSON config into a Colab secret.  
This secret will persist over kernel restarts and can be used by all of your notebooks, 
but it will not be exposed if you share your notebook with anyone.

![Screenshot of setting a Colab Secret](images/colab-secret.png)

1. Click on the key icon at the left of Google Colab to open the Secrets interface.
2. Click on "Add new secret"
3. Ensure that "Notebook access" is enabled
4. For the name, enter `APERTUREDB_JSON`
5. Paste the JSON object into the "Value" box.

### e) For Jupyter lab - Run adb config create

If there is no active configuration, the new one is used for all future ApertureDB communication but it's easy to switch between servers by activating different configuration profiles created as below:

In [ ]:
! adb config create my_adb_config

Enter aperturedb host name [localhost]: 

## 3. Connect ApertureDB Client and Server

As a demonstration that you have connected to the server successfully, let's see a summary of the database schema.  
The first time you do this, you may need to grant permission for this notebook to access your secrets.

In [ ]:
from aperturedb.Utils import Utils
from aperturedb.CommonLibrary import create_connector

# Create the connector for ApertureDB
connector = create_connector()

In [ ]:
# Use the connector to create a Utils object and print the summary
utils = Utils(connector)
utils.summary()

## 4. Ingest Our Example Cookbook Data

* [Git Repository](https://github.com/aperture-data/Cookbook)
* [More details on how to ingest](https://docs.aperturedata.io/category/manage-multimodal-datasets)

In [ ]:
! wget https://github.com/aperture-data/Cookbook/blob/quick_start/notebooks/load_cookbook_data.sh
! bash ingest_cookbook_data.sh

## 5. Run Queries!

### a) Find images of dishes served for breakfast

### b) Find images and information of dishes with butter and chicken in them

### c) Find similar looking dishes to a bread